GAGAS Evaluation

In [18]:
import requests
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter 
from langchain.embeddings import OpenAIEmbeddings
import pinecone
from dotenv import load_dotenv, find_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [30]:
# Data loader
def data_loader(file_path= '../prompts/Weekly_Challenge_Week_6.txt'):
    loader = TextLoader(file_path)
    documents = loader.load()

    # Chunk the data
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_documents(documents)
    return chunks

In [32]:
import time
import os
import pinecone

def create_retriever(chunks):

  # Load Pinecone API key from .env file
  load_dotenv(find_dotenv())

 # Initialize Pinecone
  pinecone.init(
    api_key=os.environ.get('PINECONE_API_KEY') or '3306f52a-a64a-46dd-b81a-0d073fb5a072',
    environment=os.environ.get('PINECONE_ENVIRONMENT') or 'gcp-starter'
  )

 # Create a new Pinecone index
  index_name = "my-index"
  pinecone.create_index(
        index_name,
        dimension=1536,
        metric='cosine'
    )
  while not pinecone.describe_index(index_name).status['ready']:
        time.sleep(1)
 # Get the Pinecone index
  index = pinecone.Index(index_name)

 # Populate Pinecone index
  for chunk in chunks:
    embeddings = OpenAIEmbeddings(chunk)
    index.upsert(ids=[chunk], vectors=embeddings)

 # Define Pinecone index as retriever to enable semantic search
  retriever = index.as_retriever()
  return retriever

In [33]:
chunks =  data_loader()
retriever = create_retriever(chunks)

Created a chunk of size 1757, which is longer than the specified 500
Created a chunk of size 1320, which is longer than the specified 500
Created a chunk of size 859, which is longer than the specified 500
Created a chunk of size 517, which is longer than the specified 500


TypeError: BaseModel.__init__() takes 1 positional argument but 2 were given